# Detectando objetos com YOLO v4 e OpenCV

## Visão geral

No notebook anterior, utilizamos o framework Darknet para realizar a detecção de objetos em imagens com o You only look once (YOLO). Entretanto, também é possível empregar o módulo cv2, que corresponde à implementação do OpenCV em Python, para executar a mesma tarefa, sem a necessidade de compilar (buildar) o framework Darknet.

## Etapa 1 - Importando as bibliotecas

In [ ]:
import cv2

print(cv2.__version__)

## Etapa 2 - Conectando com o Google Drive

In [ ]:
# Desnecessario

## Etapa 3 - Carregando os arquivos do modelo treinado

In [ ]:
import utils

# Obter pesos, configuracao da arquitetura e labels da rede neural do modelo
yolov3 = utils.carregar_yolov3()
yolov3

In [ ]:
# Carregar a rede YOLOv3 no módulo DNN (Deep Neural Network) do OpenCV
net = cv2.dnn.readNet(yolov3.config_path, yolov3.weights_path)
net

## Etapa 4 - Definindo mais configurações para a detecção

In [ ]:
import numpy as np

# Gerar cores RGB aleatorias para "colorir" o contorno da caixa delimitadora de cada label/classe detectada
# NOTE: o OpenCV trabalha com o sistema de cores BGR ao inves do RGB
colors = np.random.randint(0, 255, size=(len(yolov3.labels), 3), dtype='uint8')
colors[:5]

Como é possível ver a seguir, uma rede neural pode ter mais de uma camada de saída, ou seja, camadas que recebem conexões da camada anterior, mas não se conectam a nenhuma camada seguinte.

**ATENÇÃO:** o número de camadas de saída em uma CNN não deve ser confundido com o número de labels (classes) que o modelo é capaz de prever.

In [ ]:
# Lista com os identificadores (IDs) de todas as camadas da rede
ln = net.getLayerNames()

# Retorna os índices (baseados em 1) das camadas de saida
ln_saida_indices = net.getUnconnectedOutLayers()

# IDs das camadas de saída
ln_saida_ids = tuple(ln[i - 1] for i in ln_saida_indices)

print('Todas as camadas:', ln)
print('Camadas de saída:', ln_saida_ids)
print('Total de camadas: ' + str(len(ln)))

## Etapa 5 - Carregando a imagem onde será feita a detecção

In [ ]:
imagem = utils.carregar_imagem_cv2('dog.jpg')

In [ ]:
imagem_cp = imagem.copy()
(H, W) = imagem_cp.shape[:2]
print(f'Dimensoes da imagem: {H} x {W} px')

## Etapa 6 - Processando a imagem de entrada

Antes que a Rede Neural Convolucional (CNN) possa realizar inferências, a imagem precisa passar por um pré-processamento.
O objetivo aqui é preparar os dados de forma que a rede consiga trabalhar corretamente, e logo após realizar as deteções (ver imagem da Etapa 8). 

Um dos métodos usados para isso é o blobFromImage, que aplica duas transformações principais:

1. **Subtração da média (mean subtraction):** remove valores médios de cor da imagem, reduzindo problemas relacionados à variação de iluminação. Isso garante que a CNN foque mais nos padrões visuais (formas, bordas, texturas) e menos nas diferenças de brilho.

2. **Redimensionamento (resizing):** ajusta a largura e altura da imagem para o tamanho de entrada exigido pela CNN.

Dessa forma, todas as imagens entram na rede com o mesmo formato, independentemente da resolução original.

> O termo *blob* é utilizado no contexto de redes neurais convolucionais para definir uma imagem que foi redimensionada e manipulada para ser utilizadas como entrada da rede

In [ ]:
import time

fator_escala = 1 / 255.0            # Fator em que a imagem sera redimensionada
tamanho_esperado_rn = (416, 416)    # Tamanho da imagem esperado pela CNN
trocar_rgb = True                   # Inverter canais RGB (o OpenCV trabalha com BGR)?
cortar_imagem = False               # Cortar imagem para se adequar ao tamanho esperado?

inicio = time.time()

# Converter imagem para formato de blob
blob = cv2.dnn.blobFromImage(
    image=imagem,
    scalefactor=fator_escala,
    size=tamanho_esperado_rn,
    swapRB=trocar_rgb,
    crop=cortar_imagem,
)

# Enviar imagem para a rede neural
net.setInput(blob=blob)

# Processar a imagem na CNN e obter as respostas das camadas de saída
# O método forward percorre a rede: processa a imagem desde as camadas de entrada até as camadas de saída
layer_outputs = net.forward(ln_saida_ids)

termino = time.time()

print("YOLO levou {:.2f} segundos".format(termino - inicio))

## Etapa 7 - Definindo as variáveis para a detecção

In [ ]:
threshold       = 0.5   # Threshold (eh de fato o objeto?): detecções com probabilidade (confiança) de ser o objeto abaixo desse percentual sao descartadas
threshold_NMS   = 0.3   # Non-Max supression (tem um objeto?): caixas com probabilidade (confiança) de ter um objeto abaixo desse percentual sao descartadas
caixas          = []
confiancas      = []        
IDclasses       = []

## Etapa 8 - Realizando a predição

A detecção retornada por uma camada de saída da rede YOLO é codificada em uma matriz cujas linhas correspondem as caixas delimitadoras e as colunas a coeficientes de probabilidade, como mostrado na imagem a seguir

![Image size function](./img/cnn-yolo.webp)

In [ ]:
# Para cada camada de saída da rede neural (a rede pode ter várias saídas)
for output in layer_outputs:
    # Para cada detecção feita nessa camada
    for detection in output:
        # Os primeiros 5 valores de detection são: [pc, bx, by, bw, bh]
        # A partir da posição 5 temos as probabilidades de cada classe
        scores = detection[5:]
        
        # Recupera o índice da classe com maior probabilidade
        classeID = np.argmax(scores)
        
        # Recupera o valor da confiança (probabilidade de realmente ser esse objeto)
        confianca = scores[classeID]

        # Só continua se a confiança for maior que o limiar definido
        if confianca > threshold:
            # print("scores: " + str(scores))
            # print("classe mais provavel: " + str(classeID))

            # Construção da caixa delimitadora ao redor do objeto detectado
            # detection[0:4] = [centerX, centerY, width, height], normalizados entre 0 e 1
            # Multiplicamos por [W, H, W, H] para escalar para o tamanho real da imagem
            caixa = detection[0:4] * np.array([W, H, W, H])
            (centerX, centerY, width, height) = caixa.astype("int")

            # Conversão de coordenadas: de centro (centerX, centerY) para canto superior esquerdo (x, y)
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))

            # Adiciona a caixa calculada à lista de caixas
            caixas.append([x, y, int(width), int(height)])

            # Adiciona a confiança dessa detecção à lista de confiabilidades
            confiancas.append(float(confianca))

            # Adiciona o ID da classe detectada à lista de classes
            IDclasses.append(classeID)


In [ ]:
print(confiancas), len(confiancas)

In [ ]:
print(IDclasses), len(IDclasses)

## Etapa 9 - Aplicando a Non-maxima Suppresion (NMS)

In [ ]:
objetos = cv2.dnn.NMSBoxes(caixas, confiancas, threshold, threshold_NMS)
objetos

In [ ]:
# Visualizar probabilidade dos objetos detectados
{int(obj): confiancas[obj]*100 for obj in objetos}

## Etapa 10 - Mostrando o resultado da deteção na imagem

In [ ]:
if len(objetos) > 0:
    for i in objetos.flatten():
        (x, y) = (caixas[i][0], caixas[i][1])
        (w, h) = (caixas[i][2], caixas[i][3])

        objeto = imagem_cp[y: y+h, x:x+w]

        cor = [int(c) for c in colors[IDclasses[i]]]

        cv2.rectangle(imagem, (x,y), (x+w, y+h), cor, 2)
        texto = "{}: {:.4f}".format(yolov3.labels[IDclasses[i]], confiancas[i])
        cv2.putText(imagem, texto, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, cor, 2)

cv2.imwrite('resultado.jpg', imagem)

utils.mostrar_imagem_cv2(imagem)